In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import os
import matplotlib.pyplot as plt
import glob
import pandas as pd

# Anomaly Solutions
<!-- requirement: projects/trip-subset -->
<!-- requirement: small_data/nycp.csv -->
<!--exclude from index-->

In [ ]:
datadir = 'projects/tripdata' if os.path.isdir('projects/tripdata') else 'projects/trip-subset/'

In [ ]:
def load_counts(fn):
    df = pd.read_csv(fn, compression='zip')
    counts = df['starttime'].str.split(' ', 1).apply(lambda x: x[0]).value_counts()
    if '-' in counts.index[0]:
        counts.index = pd.to_datetime(counts.index, format='%Y-%m-%d')
    else:
        counts.index = pd.to_datetime(counts.index, format='%m/%d/%Y')
    return counts.sort_index()

fns = glob.glob(os.path.join(datadir,'[0-9][0-9][0-9][0-9][0-9][0-9]-citibike-tripdata.zip'))
counts = pd.concat([load_counts(fn) for fn in sorted(fns)])

1. Lets practice using the apply function!  Write a function which filters the dataframe for rides that started in a particular hour of the day.

In [ ]:
df = pd.read_csv(os.path.join(datadir,'201307-citibike-tripdata.zip'), compression='zip')

In [ ]:
def filter_by_hour(df, hour):
    hour_str = str(hour).zfill(2)
    return df[df['starttime'].apply(lambda x : x.split(' ')[1][:2]) == hour_str]

In [ ]:
filter_by_hour(df, 2).head()

1. Growth is often exponential. Use a transformer to model exponential, instead of quadratic, growth, as a feature. How do the two models compare?

In [ ]:
from sklearn import base
import numpy as np

class QuadBackground(base.BaseEstimator, base.TransformerMixin):
            
    def fit(self, X, y=None):
        self.X0 = X[0]
        return self
    
    def transform(self, X):
        days = (X - self.X0).days
        return np.c_[days, days**2]
    
class ExpBackground(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, tau):
        self.tau = float(tau)
        
    def fit(self, X):
        self.X0 = X[0]
        return self
    
    def transform(self, X):
        days = (X - self.X0).days
        return np.c_[days, np.exp(days/self.tau)]

In [ ]:
from sklearn import metrics
def rmse(y, y_pred):
    return np.sqrt(metrics.mean_squared_error(y, y_pred))

In [ ]:
from sklearn.linear_model import LinearRegression

X_trans =  QuadBackground().fit_transform(counts.index)
lr = LinearRegression()
lr.fit(X_trans, counts)

X_trans_exp = ExpBackground(200).fit_transform(counts.index)
lr_exp = LinearRegression()
lr_exp.fit(X_trans_exp, counts)

plt.plot(counts.index, counts)
plt.plot(counts.index, lr.predict(X_trans), label='quadratic')
plt.plot(counts.index, lr_exp.predict(X_trans_exp), label='exp')
plt.legend()
print("RMSE Quad = {:.5f}".format(rmse(counts, lr.predict(X_trans))))
print("RMSE Exp = {:.5f}".format(rmse(counts, lr_exp.predict(X_trans_exp))))

1. Create a pipeline with the previous `QuadBackground` model.

In [ ]:
from sklearn.pipeline import Pipeline
quad_pipe = Pipeline([
    ('quad', QuadBackground()),
    ('lr', LinearRegression())
])
quad_pipe.fit(counts.index, counts)

1. Account for the monthly seasonality.  Examine how ridership varies over the month.  Develop a model to account for this.  How much does this improve the RMSE?

Lets first load in the old model so we can compare results

In [ ]:
from sklearn.pipeline import FeatureUnion

class FourierComponents(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, period):
        self.period = period
    
    def fit(self, X, y=None):
        self.X0 = X[0]
        return self
    
    def transform(self, X):
        dt = (X - self.X0).days * 2 * np.pi / self.period
        return np.c_[np.sin(dt), np.cos(dt)]
    
union = FeatureUnion([('fourier', FourierComponents(365)),
                      ('growth', QuadBackground())])

pipe = Pipeline([('union', union),
                 ('lr', LinearRegression())])
pipe.fit(counts.index, counts.values)
plt.plot(counts.index, counts, counts.index, pipe.predict(counts.index))
print("RMSE: {}".format(rmse(counts.values, pipe.predict(counts.index))))
print("R^2: {}".format(pipe.score(counts.index, counts.values)))

Now we can add in monthly seasonality

In [ ]:
union = FeatureUnion([('fourier', FourierComponents(365)),
                      ('fourier_months', FourierComponents(30.5)),
                      ('growth', QuadBackground())])

pipe = Pipeline([('union', union),
                 ('lr', LinearRegression())])
pipe.fit(counts.index, counts.values)
plt.plot(counts.index, counts, counts.index, pipe.predict(counts.index))
print("RMSE: {}".format(rmse(counts.values, pipe.predict(counts.index))))
print("R^2: {}".format(pipe.score(counts.index, counts.values)))

It looks like we do a bit better!

1. Are there any observable (maybe with a Fourier Transform) frequencies in our residual? Should there be?


Lets first load in the final model

In [ ]:
class DayofWeek(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def day_vector(self, day):
        v = np.zeros(7)
        v[day] = 1
        return v
    
    def transform(self, X):
        return np.stack(self.day_vector(d) for d in X.dayofweek)
union = FeatureUnion([('fourier', FourierComponents(365)),
                      ('dayofweek', DayofWeek()),
                      ('growth', QuadBackground())])

pipe = Pipeline([('union', union),
                 ('lr', LinearRegression(fit_intercept=False))])
pipe.fit(counts.index, counts)

residuals = counts - pipe.predict(counts.index)
plt.plot(residuals)
plt.title("residuals");

Now we can Fourier transform our residuals

In [ ]:

fft_counts = np.fft.fft(residuals - residuals.mean())
yrs = (residuals.index[-1] - residuals.index[0]).days / 365.

plt.plot(1.0*np.arange(len(fft_counts)) / yrs, np.abs(fft_counts)**2)
plt.xlabel('Freq (1/yrs)')
plt.xlim(0, 50);

Clearly there are still some frequencies there.

1. Try removing features one at a time and see how it affects the RMSE.

Lets remove these one by one and make a bar chart of the RMSE with the labels being the removed feature

In [ ]:
features = [('date', QuadBackground()),
          ('fourier-y', FourierComponents(365)),
          ('fourier-2', FourierComponents(365/2.)),
          ('fourier-m', FourierComponents(365/12.)),
          ('fourier-8', FourierComponents(365/8.)),
          ('dayofweek', DayofWeek())]

results = []
left_out = []
for i in range(len(features)):
    union = FeatureUnion(features[:i] + features[i+1:])
    left_out.append(features[i][0])
    pipe = Pipeline([('union', union),
                     ('lr', LinearRegression())])
    pipe.fit(counts.index, counts.values)
    results.append(
        np.sqrt(metrics.mean_squared_error(counts, 
                                           pipe.predict(counts.index)))
    )
pd.DataFrame({'removed': left_out, 'rmse': results}).plot.bar(x='removed');

1. Make a plot showing the percentage of outliers detected as a function of the cutoff.  

In [ ]:
union = FeatureUnion([('date', QuadBackground()),
                      ('fourier-y', FourierComponents(365)),
                      ('fourier-2', FourierComponents(365/2.)),
                      ('fourier-m', FourierComponents(365/12.)),
                      ('fourier-8', FourierComponents(365/8.)),
                      ('dayofweek', DayofWeek())])
pipe = Pipeline([('union', union),
                 ('lr', LinearRegression())])
pipe.fit(counts.index, counts.values)
z = residuals / residuals.std()

In [ ]:
def perc_z(upper, lower):
    upper_num = z[z > upper].shape[0]
    lower_num = z[z < lower].shape[0]
    return float(upper_num + lower_num)/ z.shape[0]

up, low = np.meshgrid(np.arange(0, 5, .5), np.arange(0, -5, -.5))
df = pd.DataFrame({"upper": up.ravel(), "lower": low.ravel()})
df['perc'] = df.apply(lambda x : perc_z(**x), axis=1)

In [ ]:
sns.heatmap(df.pivot("upper", "lower", "perc"));

2. We have been looking at only aggregate data so far.  Instead, look at the ridership at a particular station, and try to detect anomalies in its data.  Do these anomalies line up with the system-wide anomalies or not?

Here we will use the windowed data model.  First we will need to load in the data keeping the station information as well.

In [ ]:
def load_df(fn):
    return pd.read_csv(fn, compression='zip')[['start station id', 'starttime']]

fns = glob.glob(os.path.join(datadir,'[0-9][0-9][0-9][0-9][0-9][0-9]-citibike-tripdata.zip'))
full_df = pd.concat([load_df(fn) for fn in sorted(fns)])

In [ ]:
full_df.head()

Now we can write a function to process the counts (similar to before) that takes the full dataframe, filters it, and processes the counts.

Unfortunately, we will have to be a bit more careful about how we format the time index because we have loaded different files together.

In [ ]:
def filter_date(d):
    if '-' in d:
        return d
    else:
        month, day, year = d.split("/")
        return "{}-{}-{}".format(year, month, day)

def process_counts(df, station_id):
    df = df[df['start station id'] == station_id]
    counts = df['starttime'].str.split(' ', 1).apply(lambda x: x[0]).value_counts()
    counts.index = counts.index.map(filter_date)
    counts.index = pd.to_datetime(counts.index, format='%Y-%m-%d')
    return counts.sort_index()

Now we we can copy over our model.

In [ ]:
class IndexExtractor(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.index

class ColumnExtractor(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, cols):
        self.cols = cols
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.cols]

time_pipe = Pipeline([('index', IndexExtractor()),
                      ('features', union)])
all_union = FeatureUnion([('time_pipe', time_pipe),
                          ('columns', ColumnExtractor(['previous', 'rolling']))])
lr_pipe = Pipeline([('all_union', all_union),
                    ('lr', LinearRegression())])

Now lets write a function to return the anomalies for a given station id.

In [ ]:
def anomaly_by_station(station_id):
    counts = process_counts(full_df, station_id)
    counts_df = pd.DataFrame({'counts': counts, 
                              'previous': counts.shift(1).fillna(method='bfill'),
                              'rolling': counts.rolling(window=5).mean().shift(1).fillna(method='bfill'),
                              'larger_rolling': counts.rolling(window=10).mean().shift(10).fillna(method='bfill')}) 
    lr_pipe.fit(counts_df, counts_df['counts'])
    residuals_window = counts_df['counts'] - lr_pipe.predict(counts_df)
    z_window = residuals_window / residuals_window.std()
    return {'upper': z_window[z_window > 2], 
            'lower': z_window[z_window < -3.5]}

Now we can process each station id.  Lets look at only those stations with many rides for good statistics

In [ ]:
station_counts = full_df.groupby('start station id').count()
station_counts[station_counts['starttime'] > 200000]

In [ ]:
anomalies = {id_: anomaly_by_station(id_) 
             for id_ in station_counts[station_counts['starttime'] > 200000].index.unique()}

In [ ]:
hsv = plt.get_cmap('hsv')
colors = hsv(np.linspace(0, 1, len(anomalies)))
for i, (station, anomaly) in enumerate(anomalies.items()):
    for ser in ['upper', 'lower']:
        plt.scatter(anomaly[ser].index, 
                    anomaly[ser], 
                    s=10, 
                    label=station,
                    color=colors[i])      

We can leave the interpretation up to you.

## Anomaly Lab

In [ ]:
counts_df = pd.DataFrame({'counts': counts, 'previous': counts.shift(1).fillna(method='bfill'),
                          'rolling': counts.rolling(window=5).mean().shift(1).fillna(method='bfill')})


In [ ]:
weather = pd.read_csv("small_data/nycp.csv")
#here, we'll even give you a nice datetime object to work with
weather["DT"] = pd.to_datetime(weather["DATE"].apply(str), format="%Y%m%d")

In [ ]:
weather_counts = counts_df.join(weather.set_index("DT"))
weather_counts['avg'] = (weather_counts['TMAX'] + weather_counts['TMIN'])/2.0
weather_counts.head()

In [ ]:
time_pipe = Pipeline([('index', IndexExtractor()),
                      ('features', union)])
all_union = FeatureUnion([('time_pipe', time_pipe),
                          ('columns', ColumnExtractor(['previous', 'rolling']))])

full_union = FeatureUnion([('counts', all_union),
                          ('weather', ColumnExtractor(['avg']))])
lr_pipe_full = Pipeline([('full_union', full_union),
                    ('lr', LinearRegression())])

In [ ]:
lr_pipe_full.fit(weather_counts, weather_counts['counts'])

In [ ]:
print("RMSE: {}".format(rmse(lr_pipe_full.predict(weather_counts), weather_counts['counts'])))
print("R^2: {}".format(lr_pipe_full.score(weather_counts, weather_counts['counts'])))

In [ ]:
lr_pipe.fit(weather_counts, weather_counts['counts'])
print("RMSE: {}".format(rmse(lr_pipe.predict(weather_counts), weather_counts['counts'])))
print("R^2: {}".format(lr_pipe.score(weather_counts, weather_counts['counts'])))

Looks like we are doing better here!  (can you see if any other features make even more of a difference?)

Now lets take a look at how this varies as a function of temperature.  We are assuming a linear model, thus we will need to pull out the coefficient which corresponds to the weather feature.  This will be our last feature.

In [ ]:
lr_pipe_full.steps[1][1].coef_[-1]

Looks like its something like 200.  Lets see if this plays out in the data.  Lets take the ridership data and look at the average ridership as a function of temperature

In [ ]:
weather_counts.groupby('avg')['counts'].mean().plot();

It looks like its a fairly linear feature up until about 70 degrees, but then it gets a bit hot and people stop riding.  We could do some feature engineering to account for this.

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*